### Preprocesamiento temporal y generación de clips (UCF-Crime)

Este notebook implementa la **Fase 3: Preparación de los datos**, transformando el conjunto de videos
definido en la fase de EDA en una representación **a nivel de clip** mediante segmentación temporal
con ventana deslizante.

**Entradas (EDA):**
- `processed/index_videos_train.csv`
- `processed/index_videos_val.csv`
- `processed/index_videos_test.csv`

**Salidas:**
- `processed/index_clips.csv`
- `processed/video_metadata.csv`
- `processed/video_errors.csv`


# 0.Cargar librerias y rutas.


In [1]:
from pathlib import Path
import os
import json
import pandas as pd
import numpy as np
import cv2

SEED = 42
rng = np.random.default_rng(SEED)

PROJECT_ROOT = Path(os.environ.get("TESIS_ROOT", Path.cwd())).resolve()

PROCESSED_DIR = PROJECT_ROOT / "processed"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
print("PROJECT_ROOT:", PROJECT_ROOT)

PROJECT_ROOT: /home/DIINF/dvaldes/tesis


In [2]:
train_csv = PROCESSED_DIR / "index_videos_train.csv"
val_csv   = PROCESSED_DIR / "index_videos_val.csv"
test_csv  = PROCESSED_DIR / "index_videos_test.csv"

assert train_csv.exists(), f"No existe: {train_csv}"
assert val_csv.exists(),   f"No existe: {val_csv}"
assert test_csv.exists(),  f"No existe: {test_csv}"

train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)
test_df  = pd.read_csv(test_csv)

train_df["split"] = "train"
val_df["split"]   = "val"
test_df["split"]  = "test"

df_videos = pd.concat([train_df, val_df, test_df], ignore_index=True)

print("Videos totales:", len(df_videos))
display(df_videos.head(3))
print(df_videos["split"].value_counts())
print(df_videos["y"].value_counts())


Videos totales: 700


,path_abs,path_rel,y,categoria,source_folder,duration_s,split
0,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,Training-Normal-Videos-Part-1/Normal_Videos477...,0,Normal,Training-Normal-Videos-Part-1,67.200000,train
1,/home/DIINF/dvaldes/tesis/UCF_Crime/Anomaly-Vi...,Anomaly-Videos-Part-3/Robbery/Robbery112_x264.mp4,1,Robbery,Anomaly-Videos-Part-3/Robbery,45.733333,train
2,/home/DIINF/dvaldes/tesis/UCF_Crime/Anomaly-Vi...,Anomaly-Videos-Part-1/Assault/Assault011_x264.mp4,1,Assault,Anomaly-Videos-Part-1/Assault,76.266667,train


split
train    490
test     106
val      104
Name: count, dtype: int64
y
0    350
1    350
Name: count, dtype: int64


# 1.Estandarización y validación de columnas

Se unifican los nombres de columnas provenientes del EDA a un esquema interno común, validando la
presencia de las variables mínimas requeridas (`path`, `y`, `split`). Adicionalmente, se verifica que
todas las rutas de video existan en disco antes de continuar con el preprocesamiento.


In [3]:
COLUMN_MAP = {
    "path_abs": "path",
    "y": "y",
    "split": "split",
    "categoria": "category",
}

missing_src = set(COLUMN_MAP.keys()) - set(df_videos.columns)
assert not missing_src, f"Faltan columnas esperadas desde EDA: {missing_src}"

# Renombrar a esquema interno estándar
df_videos = df_videos.rename(columns=COLUMN_MAP)

# Validar esquema mínimo interno
required_cols = {"path", "y", "split"}
missing = required_cols - set(df_videos.columns)
assert not missing, f"Faltan columnas en esquema interno: {missing}"

# Normalizar path
df_videos["path"] = df_videos["path"].astype(str)
df_videos["exists"] = df_videos["path"].apply(lambda p: Path(p).exists())

print("Paths existentes:", df_videos["exists"].mean())

df_missing = df_videos[~df_videos["exists"]].copy()
if len(df_missing) > 0:
    display(df_missing.head(10))


display(df_videos.head())
print(df_videos.columns.tolist())



Paths existentes: 1.0


,path,path_rel,y,category,source_folder,duration_s,split,exists
0,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,Training-Normal-Videos-Part-1/Normal_Videos477...,0,Normal,Training-Normal-Videos-Part-1,67.200000,train,True
1,/home/DIINF/dvaldes/tesis/UCF_Crime/Anomaly-Vi...,Anomaly-Videos-Part-3/Robbery/Robbery112_x264.mp4,1,Robbery,Anomaly-Videos-Part-3/Robbery,45.733333,train,True
2,/home/DIINF/dvaldes/tesis/UCF_Crime/Anomaly-Vi...,Anomaly-Videos-Part-1/Assault/Assault011_x264.mp4,1,Assault,Anomaly-Videos-Part-1/Assault,76.266667,train,True
3,/home/DIINF/dvaldes/tesis/UCF_Crime/Testing_No...,Testing_Normal_Videos_Anomaly/Normal_Videos_93...,0,Normal,Testing_Normal_Videos_Anomaly,59.466667,train,True
4,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,Training-Normal-Videos-Part-2/Normal_Videos694...,0,Normal,Training-Normal-Videos-Part-2,732.400000,train,True


['path', 'path_rel', 'y', 'category', 'source_folder', 'duration_s', 'split', 'exists']


# 2. Extracción y validación de metadatos por video

Se define una función para extraer metadatos básicos de cada video (FPS, número de frames, resolución y
duración), validando además que el archivo exista y pueda ser abierto correctamente. Los videos que no
cumplen estas condiciones se marcan como inválidos para su posterior análisis.


In [4]:
def get_video_meta(video_path: str) -> dict:
    p = Path(video_path)
    if not p.exists():
        return {"ok": False, "error": "path_not_found"}

    cap = cv2.VideoCapture(str(p))
    try:
        if not cap.isOpened():
            return {"ok": False, "error": "cv2_cannot_open"}

        fps = cap.get(cv2.CAP_PROP_FPS)
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if fps is None or fps <= 0:
            return {"ok": False, "error": "invalid_fps", "fps": float(fps) if fps else None, "n_frames": n_frames}

        duration_s = (n_frames / fps) if n_frames > 0 else np.nan

        return {
            "ok": True,
            "fps": float(fps),
            "n_frames": int(n_frames),
            "width": int(w),
            "height": int(h),
            "duration_s": float(duration_s) if np.isfinite(duration_s) else np.nan,
        }
    finally:
        cap.release()


# 3.Construcción de metadatos y registro de errores

Se aplica la extracción de metadatos a todos los videos del conjunto, generando un registro estructurado
de videos válidos y un archivo separado con aquellos que presentan errores. Estos artefactos permiten
auditar la calidad del dataset antes de la segmentación temporal.


In [5]:
meta_rows = []
error_rows = []

for _, row in df_videos.iterrows():
    meta = get_video_meta(row["path"])

    base = {
        "split": row["split"],
        "y": int(row["y"]),
        "path": row["path"],
        "category": row.get("category", None),
        "source_folder": row.get("source_folder", None),
        "path_rel": row.get("path_rel", None),
    }
    ok = (
        meta.get("ok", False)
        and meta.get("fps") is not None
        and meta.get("fps") > 0
        and meta.get("n_frames") is not None
        and meta.get("n_frames") > 0
    )

    if ok:
        meta_rows.append({**base, **meta})
    else:
        meta_rows.append({**base, **meta})  # opcional: conservar metadata
        error_rows.append({
            **base,
            **meta,
            "error_reason": "invalid_fps_or_n_frames"
        })

df_meta = pd.DataFrame(meta_rows)
df_err  = pd.DataFrame(error_rows)

meta_out = PROCESSED_DIR / "video_metadata.csv"
err_out  = PROCESSED_DIR / "video_errors.csv"

df_meta.to_csv(meta_out, index=False)
df_err.to_csv(err_out, index=False)

print("OK videos:", len(df_meta) - len(df_err))
print("Errores:", len(df_err))

if len(df_err) > 0:
    display(df_err.head(10))


OK videos: 700
Errores: 0


# 4.Segmentación temporal y generación del índice de clips

Se segmenta cada video válido en clips temporales mediante una ventana deslizante de longitud fija y
solapamiento definido. Para cada clip se registran los frames de inicio y término, junto con la
información necesaria para preservar la etiqueta y la partición original. El resultado es un índice
estructurado a nivel de clip que será utilizado en las fases posteriores del pipeline.


In [6]:
# Parámetros de segmentación temporal (ajústalos a tu diseño experimental)
CLIP_LEN = 32          # frames por clip (ej: 16 o 32)
STRIDE = CLIP_LEN // 2 # 50% overlap
MAX_CLIPS_PER_VIDEO = None  # control de costo

print("CLIP_LEN:", CLIP_LEN, "STRIDE:", STRIDE, "MAX_CLIPS_PER_VIDEO:", MAX_CLIPS_PER_VIDEO)

CLIP_LEN: 32 STRIDE: 16 MAX_CLIPS_PER_VIDEO: None


In [7]:
def build_clips_for_video(n_frames: int, clip_len: int, stride: int):
    if n_frames < clip_len:
        return []
    starts = range(0, n_frames - clip_len + 1, stride)
    return [(s, s + clip_len) for s in starts]

clip_rows = []

for _, row in df_meta.iterrows():
    clips = build_clips_for_video(
        n_frames=int(row["n_frames"]),
        clip_len=CLIP_LEN,
        stride=STRIDE
    )

    for clip_idx, (s, e) in enumerate(clips):
        clip_rows.append({
            "split": row["split"],
            "y": int(row["y"]),
            "category": row.get("category", None),
            "path": row["path"],
            "clip_idx": int(clip_idx),
            "start_frame": int(s),
            "end_frame": int(e),
            "clip_len": int(CLIP_LEN),
            "stride": int(STRIDE),
            "fps": float(row["fps"]),
            "n_frames": int(row["n_frames"]),
        })

df_clips = pd.DataFrame(clip_rows)

clips_out = PROCESSED_DIR / "index_clips.csv"
df_clips.to_csv(clips_out, index=False)

print("Total clips:", len(df_clips))
print("\nClips por split:")
print(df_clips["split"].value_counts())

print("\nBalance global (y):")
print(df_clips["y"].value_counts())

print("\nBalance por split (split x y):")
print(pd.crosstab(df_clips["split"], df_clips["y"]))

display(df_clips.head(5))



Total clips: 145356

Clips por split:
split
train    106527
val       19793
test      19036
Name: count, dtype: int64

Balance global (y):
y
0    73870
1    71486
Name: count, dtype: int64

Balance por split (split x y):
y          0      1
split              
test    9141   9895
train  54009  52518
val    10720   9073


,split,y,category,path,clip_idx,start_frame,end_frame,clip_len,stride,fps,n_frames
0,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,0,0,32,32,16,30.0,2016
1,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,1,16,48,32,16,30.0,2016
2,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,2,32,64,32,16,30.0,2016
3,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,3,48,80,32,16,30.0,2016
4,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,4,64,96,32,16,30.0,2016
